
# Adverse drug reaction identification and extraction from natural language text using Topic modeling

## Install required packages

In [1]:
# Run in python console
import nltk; 
nltk.download('stopwords')

# Run in terminal or command prompt
#python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spacy==2.2.0

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\HP\anaconda3\python.exe' 'C:\Users\HP\anaconda3\lib\site-packages\pip' install --ignore-installed --no-user --prefix 'C:\Users\HP\AppData\Local\Temp\pip-build-env-uwa46p02\normal' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- 'preshed<3.1.0,>=3.0.2' 'cymem<2.1.0,>=2.0.2' 'cython>=0.25' 'thinc<7.2.0,>=7.1.1' 'murmurhash<1.1.0,>=0.28.0' wheel
       cwd: None
  Complete output (215 lines):
    Using cached preshed-3.0.5-cp38-cp38-win_amd64.whl (112 kB)
    Using cached cymem-2.0.5-cp38-cp38-win_amd64.whl (36 kB)
    Using cached Cython-0.29.22-cp38-cp38-win_amd64.whl (1.7 MB)
    Using cached thinc-7.1.1.tar.gz (1.9 MB)
    Using cached murmurhash-1.0.5-cp38-cp38-win_amd64.whl (21 kB)
    Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
    Using cached blis-0.4.1-cp38-cp38-win_amd64.whl (5.0 MB)
    Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
    Using ca

  Using cached spacy-2.2.0.tar.gz (5.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started


    copying thinc\neural\_classes\rnn.py -> build\lib.win-amd64-3.8\thinc\neural\_classes
    copying thinc\neural\_classes\selu.py -> build\lib.win-amd64-3.8\thinc\neural\_classes
    copying thinc\neural\_classes\softmax.py -> build\lib.win-amd64-3.8\thinc\neural\_classes
    copying thinc\neural\_classes\static_vectors.py -> build\lib.win-amd64-3.8\thinc\neural\_classes
    copying thinc\neural\_classes\__init__.py -> build\lib.win-amd64-3.8\thinc\neural\_classes
    creating build\lib.win-amd64-3.8\thinc\extra\_vendorized
    copying thinc\extra\_vendorized\keras_datasets.py -> build\lib.win-amd64-3.8\thinc\extra\_vendorized
    copying thinc\extra\_vendorized\keras_data_utils.py -> build\lib.win-amd64-3.8\thinc\extra\_vendorized
    copying thinc\extra\_vendorized\keras_generic_utils.py -> build\lib.win-amd64-3.8\thinc\extra\_vendorized
    copying thinc\extra\_vendorized\__init__.py -> build\lib.win-amd64-3.8\thinc\extra\_vendorized
    creating build\lib.win-amd64-3.8\thinc\ext

In [4]:
pip install gensim


  Installing backend dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


## Importing libaries

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Import dataset

In [84]:
# Import Dataset
df = pd.read_csv('Mirena_IUD.csv', encoding='cp1252',error_bad_lines=False)
df.head()

,Title,Time,reporttype,Reviews,Unnamed: 4
0,Rated for Birth Control Report,Posted 6 months ago (8/19/2020),Report,I got the Mirena in Feb 2020. The insertion wa...,NaN
1,Rated for Birth Control Report,Posted 6 months ago (8/19/2020),Report,This is 2nd time I have had the Mirena. Total ...,NaN
2,Rated for Birth Control Report,Posted 6 months ago (8/19/2020),Report,The biggest joke known to mankind. Doctor gave...,NaN
3,Rated for Birth Control Report,Posted 6 months ago (8/14/2020),Report,I’ve had mirena for exactly one month now and ...,NaN
4,Rated for Endometriosis Report,Posted 7 months ago (8/6/2020),Report,"I have a history of gushing, painful periods w...",NaN


# Data pre-processing steps-

### Drop unwanted columns

In [85]:
df.drop(columns = ['Title','Time','reporttype','Unnamed: 4'],inplace = True)
df.head()

,Reviews
0,I got the Mirena in Feb 2020. The insertion wa...
1,This is 2nd time I have had the Mirena. Total ...
2,The biggest joke known to mankind. Doctor gave...
3,I’ve had mirena for exactly one month now and ...
4,"I have a history of gushing, painful periods w..."


## Prepare stopwords list

In [86]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','use','day','months','get','years','go','time','would','like'])
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Lowercasing the text

In [87]:
df["Reviews"]=df["Reviews"].str.lower()
df.head()

,Reviews
0,i got the mirena in feb 2020. the insertion wa...
1,this is 2nd time i have had the mirena. total ...
2,the biggest joke known to mankind. doctor gave...
3,i’ve had mirena for exactly one month now and ...
4,"i have a history of gushing, painful periods w..."


### Remove digits & words+digits

In [88]:
df['Reviews']=df['Reviews'].apply(lambda x: re.sub('\w*\d\w*','', x))
df.head()

,Reviews
0,i got the mirena in feb . the insertion wasn't...
1,this is time i have had the mirena. total of ...
2,the biggest joke known to mankind. doctor gave...
3,i’ve had mirena for exactly one month now and ...
4,"i have a history of gushing, painful periods w..."


### Remove punctuations

In [89]:
#the string.punctuation in python contains the following punctuation symbols
#     !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`


#import spacy
import string
df["Reviews"] = df["Reviews"].astype(str)
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(Reviews):
    return Reviews.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["Reviews"]= df["Reviews"].apply(lambda Reviews: remove_punctuation(Reviews))
df.head(118)

,Reviews
0,i got the mirena in feb the insertion wasnt p...
1,this is time i have had the mirena total of ...
2,the biggest joke known to mankind doctor gave ...
3,i’ve had mirena for exactly one month now and ...
4,i have a history of gushing painful periods wh...
...,...
113,i had it in twice in years first one for and...
114,woah big difference just in one day taking it ...
115,the reason my girlfriend had to use the mirena...
116,have the mirena a little over years now and i...


In [90]:
# Convert to list
data = df.Reviews.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove quotes
data = [re.sub("\'"," ", sent) for sent in data]

pprint(data[:2])

['i got the mirena in feb the insertion wasnt pleasant by any means it was '
 'like instant contractions and my hands were gripping the table i was pretty '
 'uncomfortable for the rest of that evening and the next day but i was able '
 'to manage any discomfort with ibuprofen for the next couple of months i just '
 'went about my lifemy job changed covid hit so i thought i was feeling out of '
 'control because of those things until i really started paying attentioni '
 'noticed my emotions were all over the map i couldnt keep myself under '
 'control i was crying all of the time some days my boyfriend was looking at '
 'me likewhat is wrong then i noticed the severe change in my appetite i was '
 'hungry all of the time all i wanted was salt and sugar all day long my skin '
 'was a mess broken out all of the time on my face chest and back i was having '
 'no period just like they said i would but i was having pms everyday hours a '
 'day days a week with no period to show for it whic

## Word Tokenization

Tokenizing each sentence into a list of words.

In [91]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:2])


[['got', 'the', 'mirena', 'in', 'feb', 'the', 'insertion', 'wasnt', 'pleasant', 'by', 'any', 'means', 'it', 'was', 'like', 'instant', 'contractions', 'and', 'my', 'hands', 'were', 'gripping', 'the', 'table', 'was', 'pretty', 'uncomfortable', 'for', 'the', 'rest', 'of', 'that', 'evening', 'and', 'the', 'next', 'day', 'but', 'was', 'able', 'to', 'manage', 'any', 'discomfort', 'with', 'ibuprofen', 'for', 'the', 'next', 'couple', 'of', 'months', 'just', 'went', 'about', 'my', 'lifemy', 'job', 'changed', 'covid', 'hit', 'so', 'thought', 'was', 'feeling', 'out', 'of', 'control', 'because', 'of', 'those', 'things', 'until', 'really', 'started', 'paying', 'attentioni', 'noticed', 'my', 'emotions', 'were', 'all', 'over', 'the', 'map', 'couldnt', 'keep', 'myself', 'under', 'control', 'was', 'crying', 'all', 'of', 'the', 'time', 'some', 'days', 'my', 'boyfriend', 'was', 'looking', 'at', 'me', 'likewhat', 'is', 'wrong', 'then', 'noticed', 'the', 'severe', 'change', 'in', 'my', 'appetite', 'was', '

## Creating phrase model

In [92]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram

bigram_mod = gensim.models.phrases.Phraser(bigram)




trigram_mod = gensim.models.phrases.Phraser(trigram)

# See bigram example
print(bigram_mod[bigram_mod[data_words[801]]])

['chills', 'itching', 'mood_swings', 'abnormal', 'bleeding', 'cramps', 'nauseous', 'breasts', 'are', 'very', 'tender', 'and', 'back', 'pain']


In [93]:
# See trigram example
print(bigram_mod[trigram_mod[data_words[525]]])

['the', 'insertion', 'was', 'simple', 'had', 'no', 'spotting', 'or', 'bleeding', 'since', 'got', 'it', 'in', 'about', 'three', 'weeks', 'ago', 'love', 'it', 'am', 'really', 'looking_forward', 'to', 'not', 'having', 'to', 'worry', 'about', 'taking', 'pill', 'every', 'day', 'and', 'it', 'is', 'agreeing', 'with', 'my', 'body', 'great']


## Creating Bigrams and Lemmatization

In [94]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [95]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

[['get', 'mirena', 'feb', 'insertion', 'be', 'pleasant', 'mean', 'instant', 'contraction', 'hand', 'grip', 'table', 'pretty', 'uncomfortable', 'rest', 'evening', 'able', 'manage', 'discomfort', 'ibuprofen', 'next', 'couple', 'go', 'lifemy', 'job', 'change', 'covid', 'hit', 'think', 'feel', 'control', 'thing', 'really', 'start', 'pay', 'attentioni', 'notice', 'emotion', 'map', 'keep', 'control', 'crying', 'day', 'boyfriend', 'look', 'likewhat', 'wrong', 'notice', 'severe', 'change', 'appetite', 'hungry', 'want', 'salt', 'sugar', 'long', 'skin', 'mess', 'broken', 'face', 'chest', 'back', 'period', 'say', 'pm', 'everyday', 'hour', 'day', 'week', 'period', 'show', 'think', 'mess', 'mentally', 'body', 'want', 'period', 'bad', 'happen', 'slowly', 'first', 'able', 'deal'], ['mirena', 'total', 'total', 'never', 'issue', 'surf', 'get', 'side', 'random', 'period']]


### Create the Dictionary and Corpus

In [96]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])  #term document matrix

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 2), (51, 1), (52, 3), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 2), (71, 1), (72, 1)]]


In [97]:
id2word[2] #word id mapping

'attentioni'

## Term-frequency

In [98]:
corpus[:1][0][:10] ##here word id 9 appeared 2 times in 1st document

[(0, 2),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2)]

In [99]:
# Human readable format of corpus 
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 2),
  ('appetite', 1),
  ('attentioni', 1),
  ('back', 1),
  ('bad', 1),
  ('be', 1),
  ('body', 1),
  ('boyfriend', 1),
  ('broken', 1),
  ('change', 2),
  ('chest', 1),
  ('contraction', 1),
  ('control', 2),
  ('couple', 1),
  ('covid', 1),
  ('crying', 1),
  ('day', 2),
  ('deal', 1),
  ('discomfort', 1),
  ('emotion', 1),
  ('evening', 1),
  ('everyday', 1),
  ('face', 1),
  ('feb', 1),
  ('feel', 1),
  ('first', 1),
  ('get', 1),
  ('go', 1),
  ('grip', 1),
  ('hand', 1),
  ('happen', 1),
  ('hit', 1),
  ('hour', 1),
  ('hungry', 1),
  ('ibuprofen', 1),
  ('insertion', 1),
  ('instant', 1),
  ('job', 1),
  ('keep', 1),
  ('lifemy', 1),
  ('likewhat', 1),
  ('long', 1),
  ('look', 1),
  ('manage', 1),
  ('map', 1),
  ('mean', 1),
  ('mentally', 1),
  ('mess', 2),
  ('mirena', 1),
  ('next', 1),
  ('notice', 2),
  ('pay', 1),
  ('period', 3),
  ('pleasant', 1),
  ('pm', 1),
  ('pretty', 1),
  ('really', 1),
  ('rest', 1),
  ('salt', 1),
  ('say', 1),
  ('severe', 1),
  (

## Latent Dirichlet Allocation(LDA) model building

`alpha` and `eta` are hyperparameters that affect sparsity of the topics. According to the gensim docs, both defaults to 1.0/num_topics prior.
`chunksize` is the number of documents to be used in each training chunk.  `update_every` determines how often the model parameters should be updated and `passes` is the total number of training passes.

In [103]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=350, #Number of documents to load into memory at a time
                                           passes=40, #Number of passes through the entire corpus
                                           alpha='auto',
                                           per_word_topics=True)


### Output

In [104]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

#weight_gain, hair_loss, panic_attack, ectopic_pregnancy

[(0,
  '0.048*"breastfeed" + 0.032*"crampy" + 0.027*"obviously" + 0.018*"i" + '
  '0.017*"goes_away" + 0.017*"deliver" + 0.016*"site" + 0.016*"burn" + '
  '0.013*"patient" + 0.012*"date"'),
 (1,
  '0.058*"weight" + 0.047*"lose" + 0.041*"gain" + 0.023*"remove" + '
  '0.022*"acne" + 0.022*"depression" + 0.020*"hair" + 0.019*"bloated" + '
  '0.017*"anxiety" + 0.017*"mood_swing"'),
 (2,
  '0.054*"copper" + 0.027*"scream" + 0.026*"enjoy" + 0.022*"case" + '
  '0.020*"abnormal" + 0.019*"unpleasant" + 0.013*"hormone" + '
  '0.013*"contraception" + 0.012*"absolute" + 0.012*"research"'),
 (3,
  '0.021*"cycle" + 0.019*"heavy" + 0.019*"acne" + 0.016*"iud" + 0.014*"find" + '
  '0.014*"mirena" + 0.012*"back" + 0.012*"remove" + 0.011*"anxiety" + '
  '0.010*"become"'),
 (4,
  '0.024*"drink" + 0.022*"tiny" + 0.017*"february" + 0.015*"fill" + '
  '0.013*"milk_supply" + 0.013*"yay" + 0.012*"achy" + 0.012*"overnight" + '
  '0.011*"constipate" + 0.010*"working"'),
 (5,
  '0.039*"mirena" + 0.028*"period" + 

## Model Evaluation

In [105]:
# Perplexity Score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.722136925675051

Coherence Score:  0.4903921312248126


In [106]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.406618 -0.093135       1        1  66.479655
17     0.351138 -0.201325       2        1  15.712612
3      0.196531  0.161415       3        1   5.161778
1      0.179664  0.296421       4        1   4.172489
14     0.097251  0.050550       5        1   2.538829
10    -0.044259  0.006349       6        1   1.169399
8     -0.082005  0.004089       7        1   0.706309
7     -0.082797 -0.013378       8        1   0.545933
2     -0.078214 -0.006764       9        1   0.540921
6     -0.077415 -0.009997      10        1   0.447736
4     -0.090144 -0.024360      11        1   0.414327
16    -0.090609 -0.019941      12        1   0.391947
0     -0.086803 -0.018472      13        1   0.376862
9     -0.091804 -0.018062      14        1   0.312120
12    -0.084986 -0.006523      15        1   0.305407
19    -0.088948 -0.027357      16        1   0.212634
11    -0.084058 -0.020173      17        1   0.205879
13    -0.087850 -0.019252      18        1   0.181686
18    -0.081939 -0.020683      19        1   0.075922
15    -0.079371 -0.019402      20        1   0.047554, topic_info=          Term        Freq       Total Category  logprob  loglift
16         day  382.000000  382.000000  Default  30.0000  30.0000
171     remove  391.000000  391.000000  Default  29.0000  29.0000
24        feel  721.000000  721.000000  Default  28.0000  28.0000
439      heavy  271.000000  271.000000  Default  27.0000  27.0000
344       ever  177.000000  177.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2668      weep    0.005425    0.591319  Topic20  -8.4473   2.9598
2893     truly    0.005425    0.591323  Topic20  -8.4473   2.9598
2931     blind    0.005425    0.591319  Topic20  -8.4473   2.9598
3181    stitch    0.005425    0.591318  Topic20  -8.4473   2.9598
3238  collapse    0.005425    0.591318  Topic20  -8.4473   2.9598

[952 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
877       6  0.934406   abdoman
1465      2  0.173769   ability
1465      6  0.695075   ability
641       4  0.839181  ablation
641      17  0.209795  ablation
...     ...       ...       ...
991       2  0.991654       yet
838       2  0.965859     young
1145      3  0.839251       yrs
1145      5  0.064558       yrs
1145      7  0.064558       yrs

[1176 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 18, 4, 2, 15, 11, 9, 8, 3, 7, 5, 17, 1, 10, 13, 20, 12, 14, 19, 16])